## Publication: Continous Tomography Implementation


In [ ]:
from tomobase.log import logger
import numpy as np

from tomoacquire import tiltschemes
from tomobase import data, phantoms
import pandas as pd
import os 


def get_save_path(filename):
    save_path = '../data'
    return os.path.join(save_path, filename)

indices = np.linspace(1, 71, 71)
logger.info(indices)

vol = phantoms.nanocage()

tilt_angles_grs = tiltschemes.GRS(70.0, -70.0).get_angle_array(indices-1)
tilt_angles_increment = tiltschemes.Incremental(-70.0, 70.0, 2).get_angle_array(indices-1)
tilt_angles_binary = tiltschemes.Binary(70.0, -70.0, 8).get_angle_array(indices-1)
tilt_angles_binary2 = tiltschemes.Binary(70.0, -70.0, 8).get_angle_array(indices-1, conventional = True)

logger.info('GRS: {}'.format(tilt_angles_grs))
logger.info('Increment: {}'.format(tilt_angles_increment))
logger.info('Binary: {}'.format(tilt_angles_binary))
logger.info('Binary2: {}'.format(tilt_angles_binary2))
data = {
    'Index': indices,
    'GRS': tilt_angles_grs,
    'Incremental': tilt_angles_increment,
    'Binary': tilt_angles_binary,
    'Binary-Conventional': tilt_angles_binary2
}
df = pd.DataFrame(data)

### Get Tilt Series 


In [19]:

from tomobase import processes
from tomobase.data import Sinogram, Volume
import os

ts_grs = processes.project(vol, tilt_angles_grs)
ts_increment = processes.project(vol, tilt_angles_increment)

ts_grs.to_file(os.path.join(save_path, 'TiltGRS.mat'))
ts = Sinogram.from_file(os.path.join(save_path, 'TiltGRS.mat'))

ts_grs.to_file(os.path.join(save_path,'TiltIncrement.mat'))
ts = Sinogram.from_file(os.path.join(save_path, 'TiltIncrement.mat'))

volume_grs = processes.reconstruct(ts_grs,'sirt')
volume_increment = processes.reconstruct(ts, 'sirt')

volume_grs.to_file(os.path.join(save_path,'ReconGRS.rec'))
volume_increment.to_file(os.path.join(save_path,'ReconIncrement.rec'))


In [ ]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.filters import threshold_otsu

name  = 'compare-nomask'
volume = volume_increment.data
vol = volume_grs

#threshold_value = threshold_otsu(volume.flatten())
#volume_masked = np.where(volume.data > threshold_value, volume, 0)
volume_masked = volume

psnr_value = psnr(vol.data, volume_masked, data_range =1.0)
ssim_value, _ = ssim(vol.data, volume_masked, full=True, multichannel=True, data_range =1.0)
if os.path.exists(os.path.join(save_path, 'psnr_ssims.csv')):
    df = pd.read_csv(os.path.join(save_path,'psnr_ssims.csv'))
else:
    df = pd.DataFrame()
data = {
    'Name': name,
    'PSNR': psnr_value,
    'SSIM': ssim_value
}
print('PSNR: {}'.format(psnr_value))
print('SSIM: {}'.format(ssim_value))

df2 = pd.DataFrame(data, index=[0])
df = pd.concat([df, df2])
df.to_csv(os.path.join(save_path, 'psnr_ssims.csv'), index=False)

